# preprocessing

dev env colab
## Load csv file from Gdrive

In [179]:
# Mount google drive to read images from colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/DevEnv/9321

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DevEnv/9321


In [0]:
# %cd /content/drive/My Drive/DevEnv/9321
# !wget https://github.com/mysilver/COMP9321-Data-Services/raw/master/20t1/assign3/training.csv
# !wget https://github.com/mysilver/COMP9321-Data-Services/raw/master/20t1/assign3/validation.csv

# Start

In [0]:
import pandas as pd
import numpy as np
import requests
import json

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from math import sqrt
import matplotlib.pyplot as plt

In [0]:
# ref: https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None  # default='warn'

In [0]:
train_df=pd.read_csv('./training.csv')
validation_df=pd.read_csv('./validation.csv')

In [184]:
train_df

,movie_id,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,rating
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,3
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",3
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,3
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,3
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,9594,"[{""cast_id"": 1, ""character"": ""Alex / Chad Wagn...","[{""credit_id"": ""52fe450ec3a36847f80b9d6f"", ""de...",15000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 28, ""n...",NaN,"[{""id"": 2143, ""name"": ""loss of parents""}, {""id...",en,Double Impact,Jean Claude Van Damme plays a dual role as Ale...,"[{""name"": ""Stone Group Pictures"", ""id"": 3470}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1991-08-10,30102717,110.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Two brothers separated by the violence. Now to...,2
2096,4638,"[{""cast_id"": 40, ""character"": ""Nicholas Angel""...","[{""credit_id"": ""569d89b6c3a36858e5004c66"",

## Processing data


|Column|Value  |
|--|--|
|Budget|keep value, not convert|
|Genres|convert to number of genres|
|Homepage|convert to boolean (has a homepage or not)|
|original_language|keep???|
|release_date|convert to year|


[release_year_rita.png](https://i.loli.net/2020/04/16/Ul1LeRQtr6PDM3d.png)

[release_year_me](https://pic.luoxufeiyan.com/uploads/20200416145052.png)


In [0]:
# Only keep certain columns as training set

# keep_col = ['budget', 'homepage', 'original_language', 'release_date']
# conv_num_list = ['cast', 'crew', 'genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']
keep_col = ['budget', 'homepage', 'release_date']
conv_num_list = ['genres', 'spoken_languages']
Y_label = 'revenue'


In [0]:
# convert-to num
def conv_to_num(dataset, modify_col):
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        curr_len = len(json.loads(curr_str))
        dataset[modify_col][i] = curr_len
    return dataset

In [0]:
# preprocessing-homepage
def proc_homepage(dataset):
    modify_col = 'homepage'
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        # convert to Ture if movie has a homepage
        curr_len = (type(curr_str) is str)
        dataset[modify_col][i] = curr_len
    return dataset

In [0]:
# preprocessing-release_date
def proc_rls_date(dataset):
    modify_col = 'release_date'
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        # convert to year only
        curr_len = int(curr_str[:4])
        dataset[modify_col][i] = curr_len
    return dataset

In [0]:
# original_language
def proc_orig_lang(dataset):
    all_lang = list(set(dataset['original_language'].values))
    le = LabelEncoder()
    le.fit(all_lang)
    dataset['original_language'] = le.transform(dataset['original_language'])
    return dataset

In [0]:
# budget
def proc_budget(dataset):
    modify_col = 'budget'
    #norm certain cols
    y_df = dataset[modify_col]
    y_norm = (y_df - y_df.mean()) / (y_df.max() - y_df.min())
    dataset[modify_col] = y_norm
    return dataset

In [0]:
## TEst only

# proc_dataset = train_df[keep_col]
# str1 = proc_dataset['spoken_languages'][206]
# len(json.loads(str1))

In [0]:
def proc_data(dataset):
    # add Y label
    selected_col = keep_col + conv_num_list
    selected_col.append(Y_label)
    proc_dataset = dataset[selected_col]
    # proc_dataset = dataset[keep_col]

    # do preprocessing
    #convert to num
    for i in conv_num_list:
        try:
            proc_dataset = conv_to_num(proc_dataset, i)
        except:
            print(i)

    # other processing
    proc_dataset = proc_budget(proc_dataset)
    proc_dataset = proc_homepage(proc_dataset)
    proc_dataset = proc_rls_date(proc_dataset)

    #norm certain cols
    y_df = dataset[Y_label]
    y_norm = (y_df - y_df.mean()) / (y_df.max() - y_df.min())
    proc_dataset[Y_label] = y_norm
    
    return proc_dataset

In [193]:
train_set = proc_data(train_df)
vali_set = proc_data(validation_df)
train_set

,budget,homepage,release_date,genres,spoken_languages,revenue
0,0.470268,True,2009,4,2,0.940454
1,0.636057,True,2007,3,1,0.285150
2,0.491320,True,2015,3,5,0.256339
3,0.504478,True,2012,4,1,0.329605
4,0.530794,True,2012,3,1,0.042371
...,...,...,...,...,...,...
2095,-0.113943,False,1991,4,1,-0.048748
2096,-0.121838,True,2007,3,1,-0.030645
2097,-0.111311,True,2008,3,1,-0.041609
2098,-0.113943,False,2008,2,2,-0.024965


# Fit models

* https://www.pluralsight.com/guides/non-linear-regression-trees-scikit-learn

* https://github.com/wangye707/PY_DELL/blob/b066ae4bb40273b24d698d66450e4207e3eb6808/git_book-master/chapters/Decision_Tree/decisiontree_regressor.py

In [0]:
keep_col = keep_col + conv_num_list

X_train = train_set[keep_col]
Y_train = train_set[Y_label]

X_valid = vali_set[keep_col]
Y_valid = vali_set[Y_label]

### decision tree

In [0]:
def dTree(X_train, Y_train):
    model_dtree = DecisionTreeRegressor()
    # dtree = DecisionTreeRegressor()
    model_dtree.fit(X_train, Y_train)

    return model_dtree

### random forest

In [0]:
def randomForest(X_train, Y_train):
    #RF model
    model_rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
    # model_rf = RandomForestRegressor(n_estimators=500)
    model_rf.fit(X_train, Y_train) 

    return model_rf

### Adaboost

In [0]:
def adaBoost(X_train, Y_train):
    model_ab = AdaBoostRegressor(n_estimators=500)
    model_ab.fit(X_train, Y_train)

    return model_ab 

### GBRT

In [0]:
def gbrt(X_train, Y_train):
    model_gbrt = GradientBoostingRegressor(n_estimators=500)
    model_gbrt.fit(X_train, Y_train)

    return model_gbrt

# Evaluate

In [199]:
model_dt = dTree(X_train, Y_train)
model_rf = randomForest(X_train, Y_train)
model_ab = adaBoost(X_train, Y_train)
model_gbrt = gbrt(X_train, Y_train)

model = model_dt

for i in range(3):
    if i ==0:
        model = model_gbrt
    if i == 1:
        model = model_rf
    if i ==2:
        model = model_ab

    print("Result for %s is:"%(model))
    # For train set
    Y_train_pred = model.predict(X_train)
    print("Train set result:")
    print('Mean squared Error: ',mean_squared_error(Y_train,Y_train_pred))
    print("Correlation score:%f"%(model.score(X_train,Y_train)))



    # For validation set
    print("Validation set result:")
    Y_valid_pred = model.predict(X_valid)
    print('Mean squared Error: ',mean_squared_error(Y_train,Y_train_pred))
    print("Correlation score:%f"%(model.score(X_train,Y_train)))


    print("\n")

Result for GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False) is:
Train set result:
Mean squared Error:  0.0014830084442258337
Correlation score:0.744886
Validation set result:
Mean squared Error:  0.012127514861620399
Correlation score:0.057647


Result for RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto